In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
train_dir = '/content/drive/MyDrive/data3a-4/training'
val_dir = '/content/drive/MyDrive/data3a-4/validation'

In [ ]:
train_dir = '/content/drive/MyDrive/data3a-4/datadata3a-2/training'
val_dir = '/content/drive/MyDrive/data3a-4/datadata3a-2/validation'

In [ ]:
img_width, img_height = 150, 150

In [ ]:
epochs = 50
batch_size= 64

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers  # Импортируем слои через layers

# data_augmentation = tf.keras.Sequential([
#     layers.RandomFlip("horizontal"),  # Переворот по горизонтали и вертикали
#     layers.RandomRotation(factor=(-0.1, 0.1)),                   # Случайное вращение на угол до 20%
#     layers.RandomZoom((-0.2, 0.2))  #, width_factor=(-0.2, 0.2))
# ])

# # Генератор данных для тренировки
# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=True
# )

# # Генератор данных для проверки
# val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers  # Импортируем слои через layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),  # Переворот по горизонтали и вертикали
    layers.RandomRotation(factor=(-0.1, 0.1)),                   # Случайное вращение на угол до 20%
    layers.RandomZoom((-0.2, 0.2))  #, width_factor=(-0.2, 0.2))
])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest'
)
# Генератор данных для проверки
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:

# Создание генераторов потоков данных
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 1796 images belonging to 4 classes.
Found 299 images belonging to 4 classes.


In [ ]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 30.8 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
tf.keras.backend.clear_session()
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='sigmoid')
    ])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',
              metrics=['accuracy'])
from tensorflow.keras.callbacks import ModelCheckpoint


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: module 'keras.api.optimizers.schedules' has no attribute 'CyclicLR'

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time

# Коллбэк для сохранения лучшей модели
checkpoint_path = '/content/drive/MyDrive/best_model_new4NewNewnew.h5'
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

class TimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        total_time = time.time() - self.start_time
        print(f"\nОбщее время обучения: {total_time//3600:.0f} ч. {(total_time%3600)//60:.0f} мин. {total_time%60:.2f} сек.")

time_callback = TimeCallback()

# Обучение модели
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=[checkpoint_callback, time_callback]  # Добавить все коллбэки
)

new

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers

tf.keras.backend.clear_session()

base_model = tf.keras.applications.EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_shape=(150, 150, 3)
)
base_model.trainable = False  # Заморозьте базовые слои


model = tf.keras.models.Sequential([
    # Первый сверточный блок
    tf.keras.layers.Conv2D(32, (3,3), padding='same', input_shape=(150,150,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    # Второй сверточный блок
    tf.keras.layers.Conv2D(64, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    # Третий сверточный блок
    tf.keras.layers.Conv2D(128, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    # Четвертый сверточный блок
    tf.keras.layers.Conv2D(256, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.GlobalAveragePooling2D(),

    # Полносвязные слои с регуляризацией
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Улучшенный оптимизатор с learning rate decay
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Колбеки для улучшения обучения
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-7
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        save_best_only=True,
        monitor='val_accuracy'
    )
]

71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
import time

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time

# Коллбэк для сохранения лучшей модели
checkpoint_path = '/content/drive/MyDrive/best_model_eeeeenew4NewNewnewweeeeeeeee.h5'
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

class TimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        total_time = time.time() - self.start_time
        print(f"\nОбщее время обучения: {total_time//3600:.0f} ч. {(total_time%3600)//60:.0f} мин. {total_time%60:.2f} сек.")

time_callback = TimeCallback()

# Обучение модели
history = model.fit(
    train_generator,
    epochs=250,
    validation_data=val_generator,
    callbacks=[checkpoint_callback, time_callback]  # Добавить все коллбэки
)
# После начального обучения разморозьте часть слоев
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 42s/step - accuracy: 0.3604 - loss: 1.6930 

29/29 ━━━━━━━━━━━━━━━━━━━━ 1455s 50s/step - accuracy: 0.3624 - loss: 1.6893 - val_accuracy: 0.2475 - val_loss: 1.8288
Epoch 2/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 31s 714ms/step - accuracy: 0.4705 - loss: 1.4561 - val_accuracy: 0.2475 - val_loss: 2.6013
Epoch 3/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 630ms/step - accuracy: 0.4576 - loss: 1.4097

29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 693ms/step - accuracy: 0.4585 - loss: 1.4088 - val_accuracy: 0.2542 - val_loss: 2.4996
Epoch 4/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 699ms/step - accuracy: 0.5177 - loss: 1.3230 - val_accuracy: 0.2475 - val_loss: 2.8549
Epoch 5/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 632ms/step - accuracy: 0.5189 - loss: 1.3022

29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 693ms/step - accuracy: 0.5186 - loss: 1.3028 - val_accuracy: 0.2910 - val_loss: 2.4897
Epoch 6/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 704ms/step - accuracy: 0.5349 - loss: 1.2753 - val_accuracy: 0.2876 - val_loss: 2.4496
Epoch 7/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 621ms/step - accuracy: 0.5306 - loss: 1.2434

...

29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 706ms/step - accuracy: 0.6135 - loss: 0.8800 - val_accuracy: 0.6622 - val_loss: 0.8736
Epoch 76/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 684ms/step - accuracy: 0.6203 - loss: 0.8723 - val_accuracy: 0.5084 - val_loss: 1.1866
Epoch 77/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 666ms/step - accuracy: 0.6066 - loss: 0.8853 - val_accuracy: 0.5217 - val_loss: 1.0109
Epoch 78/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 688ms/step - accuracy: 0.6503 - loss: 0.8548 - val_accuracy: 0.4582 - val_loss: 1.5432
Epoch 79/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 634ms/step - accuracy: 0.6119 - loss: 0.8759

29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 697ms/step - accuracy: 0.6119 - loss: 0.8759 - val_accuracy: 0.6722 - val_loss: 0.8959
Epoch 80/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 664ms/step - accuracy: 0.6360 - loss: 0.8768 - val_accuracy: 0.4783 - val_loss: 1.2982
Epoch 81/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 665ms/step - accuracy: 0.6037 - loss: 0.9029 - val_accuracy: 0.3980 - val_loss: 1.7531
Epoch 82/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 653ms/step - accuracy: 0.6220 - loss: 0.9011 - val_accuracy: 0.5552 - val_loss: 1.1152
Epoch 83/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 697ms/step - accuracy: 0.6345 - loss: 0.8341 - val_accuracy: 0.3645 - val_loss: 1.7070
Epoch 84/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 648ms/step - accuracy: 0.6250 - loss: 0.8933 - val_accuracy: 0.3712 - val_loss: 1.7862
Epoch 85/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 699ms/step - accuracy: 0.6241 - loss: 0.8844 - val_accuracy: 0.6020 - val_loss: 0.9796
Epoch 86/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 666ms/step - accuracy: 0.6304 - loss: 0.8694 - val_a

29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 714ms/step - accuracy: 0.6299 - loss: 0.8482 - val_accuracy: 0.6890 - val_loss: 0.8385
Epoch 91/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 674ms/step - accuracy: 0.6074 - loss: 0.8880 - val_accuracy: 0.4080 - val_loss: 1.9827
Epoch 92/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 662ms/step - accuracy: 0.6275 - loss: 0.8700 - val_accuracy: 0.4983 - val_loss: 1.2794
Epoch 93/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 662ms/step - accuracy: 0.6028 - loss: 0.8941 - val_accuracy: 0.5886 - val_loss: 0.8819
Epoch 94/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 673ms/step - accuracy: 0.6270 - loss: 0.8786 - val_accuracy: 0.6388 - val_loss: 0.9819
Epoch 95/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 689ms/step - accuracy: 0.6420 - loss: 0.8546 - val_accuracy: 0.4716 - val_loss: 1.5451
Epoch 96/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 658ms/step - accuracy: 0.6355 - loss: 0.8553 - val_accuracy: 0.3010 - val_loss: 3.1154
Epoch 97/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 659ms/step - accuracy: 0.6303 - loss: 0.8352 - val_a

29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 686ms/step - accuracy: 0.6373 - loss: 0.8254 - val_accuracy: 0.6990 - val_loss: 0.8225
Epoch 134/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 672ms/step - accuracy: 0.6554 - loss: 0.8195 - val_accuracy: 0.6522 - val_loss: 0.9261
Epoch 135/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 655ms/step - accuracy: 0.6409 - loss: 0.8160 - val_accuracy: 0.5418 - val_loss: 1.3976
Epoch 136/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 677ms/step - accuracy: 0.6715 - loss: 0.7965 - val_accuracy: 0.5552 - val_loss: 1.3435
Epoch 137/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 665ms/step - accuracy: 0.6469 - loss: 0.8104 - val_accuracy: 0.6087 - val_loss: 0.8634
Epoch 138/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 671ms/step - accuracy: 0.6528 - loss: 0.8135 - val_accuracy: 0.5953 - val_loss: 0.9328
Epoch 139/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 682ms/step - accuracy: 0.6260 - loss: 0.8433 - val_accuracy: 0.3244 - val_loss: 2.8199
Epoch 140/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 694ms/step - accuracy: 0.6397 - loss: 0.8303 

29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 687ms/step - accuracy: 0.6493 - loss: 0.7926 - val_accuracy: 0.7258 - val_loss: 0.8056
Epoch 180/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 676ms/step - accuracy: 0.6439 - loss: 0.8024 - val_accuracy: 0.5619 - val_loss: 1.1276
Epoch 181/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 649ms/step - accuracy: 0.6368 - loss: 0.8081 - val_accuracy: 0.5920 - val_loss: 0.9589
Epoch 182/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 694ms/step - accuracy: 0.6479 - loss: 0.8232 - val_accuracy: 0.5585 - val_loss: 1.1060
Epoch 183/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 667ms/step - accuracy: 0.6085 - loss: 0.8485 - val_accuracy: 0.4883 - val_loss: 1.5669
Epoch 184/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 689ms/step - accuracy: 0.6471 - loss: 0.8051 - val_accuracy: 0.5987 - val_loss: 0.9712
Epoch 185/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 665ms/step - accuracy: 0.6533 - loss: 0.7965 - val_accuracy: 0.5318 - val_loss: 1.5197
Epoch 186/250
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 666ms/step - accuracy: 0.6824 - loss: 0.7730 